In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
stack_dfs = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index()
panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
for key, df in panel_dfs_dict.items():
    for col in df.columns:
        if not is_numeric_dtype(df[col]):
            try:
                df[col] = pd.to_numeric(df[col], errors = "coerce")
                df[col] = df[col].fillna(0)
                df[col] = df[col].astype(int)
            except:
                pass
        else:
            df[col] = df[col].fillna("")
    df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
    df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
    df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])

/tmp/ipykernel_10327/1077166218.py:17: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_10327/1077166218.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_10327/1077166218.py:19: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_10327/1077166218.py:17: Perform

In [12]:
efwgdp = pd.read_csv("EFWGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()
efwgdp[["EFW", "GDP"]]

EFW           GDP
State Year                          
AK    1985-01-01  8.04           NaN
      1986-01-01   NaN           NaN
      1987-01-01   NaN           NaN
      1988-01-01   NaN           NaN
      1989-01-01   NaN           NaN
...                ...           ...
WY    2019-01-01  8.12  3.997140e+10
      2020-01-01  7.97  3.667550e+10
      2021-01-01   NaN  4.217620e+10
      2022-01-01   NaN  4.908060e+10
      2023-01-01   NaN  5.017160e+10

[1950 rows x 2 columns]

In [13]:
plot_dfs = {}
keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'TOTAL INCOME',
        "PROPERTY", "SPECIAL ASSESSMENTS", "PROPERTY AND SPECIAL ASSESSMENTS",  
        "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	
keys = ["1" + k for k in keys]
for key in panel_dfs_dict.keys():
    plot_dfs[key] = {}
    plot_dfs[key]["Level"] = panel_dfs_dict[key][keys].mul(10**3)
    plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
#     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
#     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
#     plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
    plot_dfs[key]["Level"]["GDP"] = efwgdp["GDP"]
    plot_dfs[key]["Level"]["Population"] = efwgdp["Population"]
    plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: x.div(plot_dfs[key]["Level"]["General Revenue"]) if is_numeric_dtype(x) else x)    
    plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: x.div(plot_dfs[key]["Level"]["GDP"]) if is_numeric_dtype(x) else x)
    plot_dfs[key]["Value of Per Capita"] = plot_dfs[key]["Level"].apply(lambda x: x.div(plot_dfs[key]["Level"]["Population"]) if is_numeric_dtype(x) else x)    
    for key in plot_dfs.keys():
        plot_dfs[key]["Level"]["EFW"] = efwgdp["EFW"]
plot_dfs

{'Local government CV': {'Level':                   General Revenue  General Revenue From Own Sources  \
  State Year                                                            
  AK    2005-01-01            850.0                              1000   
        2006-01-01           1010.0                              1000   
        2008-01-01           1020.0                              1000   
        2009-01-01           1030.0                              1000   
        2010-01-01           1130.0                                 0   
  ...                           ...                               ...   
  WY    2017-01-01              0.0                                 0   
        2018-01-01            640.0                                 0   
        2019-01-01            520.0                                 0   
        2020-01-01            710.0                                 0   
        2021-01-01            410.0                                 0   
  
                

In [14]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from homebrewedFunctions.functions import *

In [15]:
from homebrewedFunctions.functions import *

for key, p_dfs in plot_dfs.items():
    figs = {key: line_dropdown(dataframe) for key, dataframe in p_dfs.items()}
    fig = dict_of_figs_to_dropdown_fig(figs, show_fig = False,
                                    use_sliders = True)
    # fig.show()
    fig.write_html(f"outputs/LineplotAppStateFinancesAsPercentRevenuePercentGDPAndPerCapitaFigs{key}.html")

# fig = line_dropdown(pct_df)
# fig.write_html("line_dropdown.html")

In [16]:
for key, p_dfs in plot_dfs.items():
    for p_dfskey, df in p_dfs.items():
        map_figs = {}
        create_scatter_dropdown(df, 
                                filename = f"outputs/ScatterAppIncomePropertyAssessmentSalesFuelTaxesPctTotalRevenue{key}{p_dfskey}.html", 
                                show_fig = False)
        for name in df.keys():
            map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
        combined_fig = combine_map_figs(map_figs)
        combined_fig.write_html(f"outputs/AllMapsByVariableAndYear{key}{p_dfskey}.html")



In [19]:
area_keys = ["Local government amount",'State & local government amount', 'State government amount']
for key in area_keys:
    df = plot_dfs[key]["Level"]

    print(list(df.keys()))
# for component_name, components in components_dct.items():
#     figs[component_name] = {}
#     for key, df in panel_dfs.items():

#         figs[component_name][key] = {}
#         for state in states:
#             plot_df = df.loc[state].loc["1950":].reset_index()
#             plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
#             title_key = f"{component_name}: {state}"
#             px_fig = px.area(
#                 plot_df, x="DATE", y="Value", color="Name", title = title_key)
#             # fig = make_subplots(rows=1, cols=1)
#             # for trace in px_fig.data:
#             #     fig.add_trace(trace, row=1, col=1)
#             # fig.update_layout(px_fig.layout)
#             # gdp_and_tax_figs[key][title_key] = fig
#             figs[component_name][key][state] = px_fig

['General Revenue', 'General Revenue From Own Sources', 'Expenditure', 'Deficit', 'From Federal Government', 'From Local Governments', 'Taxes', 'Other Taxes', 'Current Charges', 'Other Charges', 'Individual Income', 'Corporate Income', 'Total Income', 'Property', 'Special Assessments', 'Property And Special Assessments', 'Sales And Gross Receipts', 'General Sales', 'Motor Fuel', 'Gas Supply', 'Education', 'Higher Education', 'Public Welfare', 'GDP', 'Population', 'EFW']
['General Revenue', 'General Revenue From Own Sources', 'Expenditure', 'Deficit', 'From Federal Government', 'From Local Governments', 'Taxes', 'Other Taxes', 'Current Charges', 'Other Charges', 'Individual Income', 'Corporate Income', 'Total Income', 'Property', 'Special Assessments', 'Property And Special Assessments', 'Sales And Gross Receipts', 'General Sales', 'Motor Fuel', 'Gas Supply', 'Education', 'Higher Education', 'Public Welfare', 'GDP', 'Population', 'EFW']
['General Revenue', 'General Revenue From Own Sour

In [18]:
areas = {"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
         "Taxes":

General Revenue  General Revenue From Own Sources  \
State Year                                                            
AK    2005-01-01     7.880540e+09                        5563360000   
      2006-01-01     8.920140e+09                        6764232000   
      2007-01-01     1.058208e+10                        8293106000   
      2008-01-01     1.618260e+10                       13991750000   
      2009-01-01     1.153169e+10                        9140243000   
...                           ...                               ...   
WY    2017-01-01     5.552848e+09                        2960793000   
      2018-01-01     5.602117e+09                        3238124000   
      2019-01-01     5.529665e+09                        3238589000   
      2020-01-01     6.740823e+09                        2937007000   
      2021-01-01     6.375373e+09                        3272303000   

                   Expenditure       Deficit  From Federal Government  \
State Year                                                              
AK    2005-01-01  8.055865e+09  1.753250e+08             2.312861e+09   
      2006-01-01  8.607819e+09 -3.123210e+08             2.152024e+09   
      2007-01-01  9.191744e+09 -1.390335e+09             2.283391e+09   
      2008-01-01  1.011591e+10 -6.066690e+09             2.185635e+09   
      2009-01-01  1.099183e+10 -5.398590e+08             2.386407e+09   
...                        ...           ...                      ...   
WY    2017-01-01  6.467711e+09  9.148630e+08             2.574251e+09   
      2018-01-01  6.333225e+09  7.311080e+08             2.356093e+09   
      2019-01-01  6.182318e+09  6.526530e+08             2.275666e+09   
      2020-01-01  6.382656e+09 -3.581670e+08             3.793522e+09   
      2021-01-01  7.494943e+09  1.119570e+09             3.089280e+09   

                  From Local Governments       Taxes   Other Taxes  \
State Year                                                           
AK    2005-01-01                 4319000  1858311000  9.749780e+08   
      2006-01-01                 3884000  2484422000  1.346859e+09   
      2007-01-01                 5582000  3688447000  2.518436e+09   
      2008-01-01                 5219000  8732385000  7.021186e+09   
      2009-01-01                 5038000  4955884000  3.906976e+09   
...                                  ...         ...           ...   
WY    2017-01-01                17804000  1649549000  5.412990e+08   
      2018-01-01                 7900000  1837401000  6.327880e+08   
      2019-01-01                15410000  2110704000  7.844440e+08   
      2020-01-01                10294000  1963896000  6.170630e+08   
      2021-01-01                13790000  1874876000  5.494240e+08   

                  Current Charges  Other Charges  ...  \
State Year                                        ...   
AK    2005-01-01      479803000.0    153391000.0  ...   
      2006-01-01      519470000.0    168886000.0  ...   
      2007-01-01      539866000.0    170474000.0  ...   
      2008-01-01      555151000.0    177172000.0  ...   
      2009-01-01      550563000.0    193982000.0  ...   
...                           ...            ...  ...   
WY    2017-01-01      226910000.0     82371000.0  ...   
      2018-01-01      250334000.0     80402000.0  ...   
      2019-01-01      249551000.0     97451000.0  ...   
      2020-01-01      252794000.0     95384000.0  ...   
      2021-01-01      248934000.0    105697000.0  ...   

                  Sales And Gross Receipts  General Sales  Motor Fuel  \
State Year                                                              
AK    2005-01-01                 198797000              0    39565000   
      2006-01-01                 208898000              0    42087000   
      2007-01-01                 235797000              0    39303000   
      2008-01-01                 277033000              0    41985000   
      2009-01-01                 250267000           

In [20]:
print(list(stack_dfs.keys()))

['1GENERAL REVENUE', '1INTERGOVERNMENTAL REVENUE', '1FROM FEDERAL GOVERNMENT', '1FROM STATE GOVERNMENT', '1FROM LOCAL GOVERNMENTS', '1GENERAL REVENUE FROM OWN SOURCES', '1TAXES', '1PROPERTY', '1SALES AND GROSS RECEIPTS', '1GENERAL SALES', '1SELECTIVE SALES', '1MOTOR FUEL', '1ALCOHOLIC BEVERAGE', '1TOBACCO PRODUCTS', '1PUBLIC UTILITIES', '1OTHER SELECTIVE SALES', '1INDIVIDUAL INCOME', '1CORPORATE INCOME', '1MOTOR VEHICLE LICENSE', '1OTHER TAXES', '1CHARGES AND MISCELLANEOUS GENERAL  REVENUE', '1CURRENT CHARGES', '1EDUCATION', '1INSTITUTIONS  OF HIGHER EDUCATION', '1SCHOOL LUNCH SALES (GROSS)', '1HOSPITALS', '1HIGHWAYS', '1AIR TRANSPORTATION (AIRPORTS)', '1PARKING FACILITIES', '1SEA AND INLAND PORT FACILITIES', '1NATURAL RESOURCES', '1PARKS AND RECREATION', '1HOUSING AND COMMUNITY DEVELOPMENT', '1SEWERAGE', '1SOLID WASTE MANAGEMENT', '1OTHER CHARGES', '1MISCELLANEOUS GENERAL REVENUE', '1INTEREST EARNINGS', '1SPECIAL ASSESSMENTS', '1SALE OF PROPERTY', '1OTHER GENERAL REVENUE', '1UTILITY R

In [35]:
rev = ['TAXES', 'PROPERTY', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'MOTOR VEHICLE LICENSE', 'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS', 'SPECIAL ASSESSMENTS', 'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', '5NAN', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE']
print(rev)

['TAXES', 'PROPERTY', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'MOTOR VEHICLE LICENSE', 'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS', 'SPECIAL ASSESSMENTS', 'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', '5NAN', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 